<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Predictive Maintenance using Vantage</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'><b>Y-Machine</b> is a manufacturing company that operates a large fleet of machines across multiple locations. They have been experiencing frequent machine breakdowns, which has been causing significant losses in production time and maintenance costs. To address this issue, <b>Y-Machine</b> is looking for a predictive maintenance solution that can help them identify potential machine failures before they occur, allowing them to proactively schedule maintenance and minimize downtime.</p>

<center><img src="./images/giphy.gif" alt="Machine GIF"/><</center>
<p><a href="https://giphy.com/gifs/Ykga9Kp0xT4GswQAbh">via GIPHY</a></p>

<p style='font-size:16px;font-family:Arial'>To achieve the goal of predictive maintenance, Y-Machine will be leveraging the power of <b>Teradata Vantage</b>, an advanced analytics platform. With Teradata Vantage, we can deploy machine learning algorithms through teradataml, which enable us to identify and mitigate potential machine failures before they even occur.</p>
<p style='font-size:16px;font-family:Arial'>Teradata Vantage provides us with the necessary capabilities to analyze the vast amounts of data generated by Y-Machine's machines, such as temperature, rotational speed, and torque. By processing this data and detecting anomalies or patterns, we can take proactive measures to address potential issues, preventing costly downtimes and ensuring the longevity of the machines.</p>
<p style='font-size:16px;font-family:Arial'>With Teradata Vantage, we can help Y-Machine stay ahead of the curve, providing them with cutting-edge analytics capabilities to improve the reliability and efficiency of their machines.</p>

<hr>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from teradataml import *
configure.val_install_location = 'val'
display.max_rows = 5

warnings.filterwarnings('ignore')

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Let's start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=PredictiveMaintenance.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_PredictiveMaintenance_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_PredictiveMaintenance_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Read the data from Vantage as a teradataml Dataframe</b>

In [ ]:
df = DataFrame(in_schema('DEMO_PredictiveMaintenance', 'Machine_Data'))

In [ ]:
df

<p style = 'font-size:16px;font-family:Arial'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>
<p style = 'font-size:16px;font-family:Arial'>The dataset above has ten columns, and the 'Target' and 'Failure_Type' columns are dependent columns where the Target column has 1 (Failure) and 0 (No Failure). The Failure_Type column has multiple types of failures.
</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Data Exploration</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Removing nulls and redundant columns</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the next cell, we'll remove the null values from the dataset.</p>

In [ ]:
# Print the number of rows before and after removing nulls

original_rows = df.shape[0]
print(f"Number of rows in the original DataFrame: {original_rows}")

df = df.dropna()
print(f"Number of rows after removing nulls: {df.shape[0]}")

<p style = 'font-size:16px;font-family:Arial'>From the above results, we see no null values in the dataset.</p>
<p style = 'font-size:16px;font-family:Arial'>In the next cell, we'll remove the Product_ID column as we already have a UID column as a unique identifier.</p>

In [ ]:
# Drop column Product_ID
df = df.drop(columns=['Product_ID'])

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Checking target variable distribution</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the next cell, we'll check the distribution of target variables, i.e., Target and Failure_Type.</p>

In [ ]:
# Count the occurrences of Failure_Type by Target and create a Pandas DataFrame
tdf = df.groupby('Target').assign(count=df.Failure_Type.count()).sort('count', ascending = False).to_pandas()

# Create a figure with a larger size
fig, ax = plt.subplots(figsize=(8, 6))

# Sort the DataFrame by count in descending order
tdf = tdf.sort_values(by='count', ascending=False)

# Create a bar chart of the counts by Target
ax = tdf.plot.bar(x="Target", y="count", rot=45, colormap='summer', ax=ax)

# Add the count to the top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', fontsize=10)

# Set the plot title and axis labels
ax.set_title("Failure Distribution")
ax.set_xlabel("Failure?")
ax.set_ylabel("Count")

# Add a grid to the plot
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add a legend to the plot
ax.legend(['Count'], loc='best', fontsize=12)

# Display the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The distribution here shows that the majority of the products have no failure, and a tiny number of products have some failure.</p>
<p style = 'font-size:16px;font-family:Arial'>Let's check further w.r.t. Failure_Type</p>

In [ ]:
# Count the occurrences of Failure_Type and create a Pandas DataFrame
tdf = df.groupby('Failure_Type').assign(count=df.Failure_Type.count()).sort('count', ascending = False).to_pandas()

# Create a figure with a larger size
fig, ax = plt.subplots(figsize=(8, 6))

# Sort the DataFrame by count in descending order
tdf = tdf.sort_values(by='count', ascending=False)

# Create a bar chart of the counts by Failure_Type
ax = tdf.plot.bar(x="Failure_Type", y="count", rot=45, colormap='summer', ax=ax)

# Add the count to the top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', fontsize=10)

# Set the plot title and axis labels
ax.set_title("Type of Failure Distribution")
ax.set_xlabel("Type of Failure")
ax.set_ylabel("Count")

# Add a grid to the plot
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add a legend to the plot
ax.legend(['Count'], loc='best', fontsize=12)

# Display the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The distribution here shows that the majority of the products have no failure, and a tiny number of products have different failures.</p>

<p style = 'font-size:16px;font-family:Arial'>There are two target variables: 'Target' and 'Failure_Type'. Let's check if everything is ok.</p>

In [ ]:
df_failure = df[df['Target'] == 1]
df_failure.groupby('Failure_Type').assign(count=df.Failure_Type.count()).sort('count', ascending = False).to_pandas()

<p style = 'font-size:16px;font-family:Arial'><b>Note:</b> 9 values are classified as failure in the 'Target' variable but as No Failure in the 'Failure_Type' variable. Let's check the dataset:</p>

In [ ]:
df_failure[df_failure['Failure_Type'] == 'No Failure']

<p style = 'font-size:16px;font-family:Arial'>It could go both ways, either failure or no failure. It makes sense to remove those instances since we do not know the real target here.</p>

In [ ]:
index_possible_failure = list(df_failure[df_failure['Failure_Type'] == 'No Failure'].get_values()[:, 0])
df = df.drop(labels=index_possible_failure, axis='index')

In [ ]:
df_failure = df[df['Target'] == 0]
df_failure.groupby('Failure_Type').assign(count=df.Failure_Type.count()).sort('count', ascending = False).to_pandas()

<p style = 'font-size:16px;font-family:Arial'><b>Note:</b> 18 instances are classified as Random Failures by 'Failure_Type', whereas they are classified as No failure by the 'Target'. These 18 instances are, in fact, all instances of 'Random Failures'. Let's check and remove those instances, as we do not know if they belong to the Failure class. Hence, we will end up with four types of failures since 'Random Failures' will be removed altogether.</p>

In [ ]:
df_failure[df_failure['Failure_Type'] == 'Random Failures']

In [ ]:
index_possible_failure = list(df_failure[df_failure['Failure_Type'] == 'Random Failures'].get_values()[:, 0])
df = df.drop(labels=index_possible_failure, axis='index')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Checking the correlation</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we'll check the distribution of target variables w.r.t features like torque, rotational speed, air temperature and process temperature.</p>

In [ ]:
df1 = df.to_pandas()

In [ ]:
# Set the figure size
fig, ax = plt.subplots(1, 2, figsize=(22, 8))

# Set the titles for each subplot
ax[0].set_title('Rot. Speed vs Torque wrt Failure Type (Including class no failure)')
ax[1].set_title('Rot. Speed vs Torque wrt Failure Type (Excluding class no failure)')

# Set the color palette for the plots
palette = ['#E9C0CB', '#39A692', '#976EBD', '#ACBF5C', '#DF8B4E']

# Plot the scatterplots
sns.scatterplot(data=df1, x='Rotational_speed', y='Torque', hue='Failure_Type', palette=palette, ax=ax[0])
sns.scatterplot(data=df1[df1['Target'] == 1], x='Rotational_speed', y='Torque', hue='Failure_Type',
                palette=palette[1:], ax=ax[1])


<p style = 'font-size:16px;font-family:Arial'><b>Some insights:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Power failure happens both for lower and higher rotational speed/torque. It is the type of failure with the highest rotational speed (over 2500rpm) and lowest torque (below around 15Nm). In other words, only power failures occur above and below these thresholds.</li>    
    <li>Between torques 16Nm and 41Nm, all failures are tool wear.</li>
    <li>Overstrain failures occur with torques ranging from around 47 and 68Nm) and rotational speeds from 1200 to 1500rpm approximately.</li>
    <li>The torque range is smaller for heat dissipation failures, and the rotational speed range is higher than for overstrain failures </li>
</ul>

In [ ]:
# Set the figure size
fig, ax = plt.subplots(1, 2, figsize=(22, 8))

# Set the titles for each subplot
ax[0].set_title('Process Temperature vs Air Temperature wrt Failure Type (Including class no failure)')
ax[1].set_title('Process Temperature vs Air Temperature wrt Failure Type (Excluding class no failure)')

# Set the color palette for the plots
palette = ['#E9C0CB', '#39A692', '#976EBD', '#ACBF5C', '#DF8B4E']

# Plot the scatterplots
sns.scatterplot(data=df1, x='Process_temperature', y='Air_temperature', hue='Failure_Type', palette=palette, ax=ax[0])
sns.scatterplot(data=df1[df1['Target'] == 1], x='Process_temperature', y='Air_temperature', hue='Failure_Type',
                palette=palette[1:], ax=ax[1])


<p style = 'font-size:16px;font-family:Arial'><b>Some insights:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Heat Dissipation Failure happens when Process Temperature and Air Temperature exceed 300 K.</li>
    <li>Other failures have no meaningful insights.</li>
</ul>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Data Transformation</b>
<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll use Label Encoder to convert a categorical variable to integer and numerical columns will be scaled using the ZScore function.</p>

In [ ]:
df.info()

In [ ]:
# Define the label encoders
type_encoder = LabelEncoder(values = {"L": 1, "M": 2, "H": 3}, columns = "Type", datatype = 'integer')
failure_type_encoder = LabelEncoder(values = {
                            "No Failure": 1,
                            "Heat Dissipation Failure": 2,
                            "Power Failure": 3,
                            "Overstrain Failure": 4,
                            "Tool Wear Failure": 5
                            }, 
                    columns=['Failure_Type'],
                    datatype = 'integer'
                  )

# Define the standard scaler
z_scaler = ZScore(columns = ['Air_temperature', 'Process_temperature',
                      'Rotational_speed', 'Torque', 'Tool_wear'],
            out_columns = ['Air_temperature', 'Process_temperature',
                      'Rotational_speed', 'Torque', 'Tool_wear'])

# Define the retain object
retain = Retain(columns = "Target")

In [ ]:
obj = valib.Transform(data = df,
                      label_encode = [type_encoder, failure_type_encoder],
                      zscore = z_scaler,
                      retain = retain,
                      index_columns = 'UID')
df_trans = obj.result

In [ ]:
df_trans.info()

<p style = 'font-size:16px;font-family:Arial'>As "Type" is a reserved keyword, we'll rename the column "Machine_type."<p>

In [ ]:
list_td_reserved_keywords('type')

In [ ]:
df_trans = df_trans.assign(Machine_type = df_trans.Type)
df_trans = df_trans.drop(columns=['Type'])

In [ ]:
df_trans

<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll split the transformed dataset into training and testing datasets in the ratio 80:20, and we will save the datasets into Vantage.</p>

In [ ]:
train_ratio = 0.8
df_sample = df_trans.sample(frac = [train_ratio, 1.0-train_ratio])

# Split into 2 virtual dataframes
df_train = df_sample[df_sample.sampleid==1].drop(["sampleid"], axis=1)
df_test = df_sample[df_sample.sampleid==2].drop(["sampleid"], axis=1)

In [ ]:
copy_to_sql(df_train,
            table_name = 'df_train',
            schema_name = 'demo_user',
            if_exists = 'replace')

copy_to_sql(df_test,
            table_name = 'df_test',
            schema_name = 'demo_user',
            if_exists = 'replace')

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. In Database Model Training (Binary Classification)</b>
<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll use the TD_XGBOOST function to train an xgboost model using the 'Target' column as the target variable for binary classification.
<br>
<br>
The TD_XGBoost function, eXtreme Gradient Boosting, implements the gradient-boosted decision tree designed for speed and performance. It has recently been dominating applied machine learning.
<br>
<br>
of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.
</p>

In [ ]:
# Create a table xgb_model using TD_XGBoost from Teradata
# The TD_XGBoost function partitions the data by any column, trains an XGBoost classification model with 10 trees, 
# maximum depth of 7, and 10 iterations, and saves the output to a metadata table xgb_out.
# If the table xgb_model already exists, drop it and the metadata table xgb_out before creating the new table.

query = f'''CREATE TABLE xgb_model AS (
SELECT * FROM TD_XGBoost(
ON df_train PARTITION BY ANY
OUT TABLE MetaInformationTable(xgb_out) 
USING
    ResponseColumn('Target')
    InputColumns('[3:8]')
    MaxDepth(7)
    NumBoostedTrees(10)
    ModelType('CLASSIFICATION')
    Seed(2)
    ShrinkageFactor(1)
    IterNum(10) 
    ColumnSampling(1.0) 
) AS dt) WITH DATA;
'''

try:
    eng.execute(query)
except Exception as e:
    # Drop the tables and try again if the table already exists
    eng.execute(f'DROP TABLE xgb_model;')
    eng.execute(f'DROP TABLE xgb_out;')
    eng.execute(query)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. In Database Model Scoring</b>
<p style = 'font-size:16px;font-family:Arial'>
In the next step, we'll use the TD_XGBoostPredict function to score the xgboost model trained in the previous step.</p>

In [ ]:
query = '''CREATE TABLE xgb_predict_out AS (
SELECT * FROM TD_XGBoostPredict(
ON df_test AS inputtable PARTITION BY ANY
ON xgb_model AS modeltable DIMENSION ORDER BY task_index, tree_num, iter, class_num, tree_order
USING
IdColumn('uid')
ModelType('classification')
Accumulate('target')
) AS dt
) WITH DATA;
'''

try:
    eng.execute(query)
except Exception as e:
    eng.execute('DROP TABLE xgb_predict_out;')
    eng.execute(query)

<p style = 'font-size:16px;font-family:Arial'>Next, we'll use the TD_ClassificationEvaluator function to evaluate the trained xgboost model on test data. This will let us know how well our model has performed on unseen data.</p>

In [ ]:
# Evaluate the XGBoost model's performance using TD_CLASSIFICATIONEVALUATOR

# Check if the necessary tables exist before executing the query
if not eng.has_table('xgb_predict_out'):
    print('Error: xgb_predict_out table does not exist.')
    sys.exit(1)

query = '''SELECT * from TD_CLASSIFICATIONEVALUATOR(
    ON (
        SELECT CAST("target" AS INTEGER) AS "target", prediction FROM xgb_predict_out
    ) AS InputTable
    OUT TABLE OutputTable(additional_metrics)
    USING
        Labels(0, 1)
        ObservationColumn('Target')
        PredictionColumn('Prediction')
) AS dt1
ORDER BY 1, 2, 3;
'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE additional_metrics;')
    eng.execute(query)


In [ ]:
pd.read_sql('SELECT * FROM additional_metrics;', eng)

In [ ]:
# Load data and compute confusion matrix
xgb_result = DataFrame(in_schema('demo_user', 'xgb_predict_out')).to_pandas().reset_index()
cm = confusion_matrix(xgb_result['Prediction'], xgb_result['Target'])

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Failure', 'Failure'])
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, cmap='Blues', colorbar=False)

# Add labels and annotations
plt.title('XGBoost Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(ticks=[0, 1], labels=['No Failure', 'Failure'])
plt.yticks(ticks=[0, 1], labels=['No Failure', 'Failure'])

# Add text to the plot to show the actual values of the confusion matrix
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, f'{cm[i, j]}', ha='center', va='center', color='white' if cm[i, j] > cm.max()/2 else 'black')

# Show the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above metrics show that our model performs well on the binary classification test dataset.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. In Database Model Training (Multi-Class Classification)</b>
<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll use the TD_XGBOOST function to train an xgboost model using Failure_Type as the target variable for multi-class classification.</p>

In [ ]:
query = '''
    CREATE TABLE xgb_model AS (
        SELECT * FROM TD_XGBoost (
            ON df_train PARTITION BY ANY
            OUT TABLE MetaInformationTable(xgb_out) 
            USING
                ResponseColumn('Failure_Type')
                InputColumns('[3:8]')
                MaxDepth(7)
                NumBoostedTrees(10)
                ModelType('CLASSIFICATION')
                Seed(2)
                ShrinkageFactor(0.9)
                IterNum(10)
                ColumnSampling(1) 
        ) AS dt
    ) WITH DATA;
'''

try:
    eng.execute(query)
except Exception as e:
    eng.execute('DROP TABLE xgb_model;')
    eng.execute('DROP TABLE xgb_out;')
    eng.execute(query)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>8. In Database Model Scoring</b>
<p style = 'font-size:16px;font-family:Arial'>In the next step, we'll use the TD_XGBoostPredict function to score the xgboost model trained in the previous step.</p>

In [ ]:
query = '''
    CREATE TABLE xgb_predict_out AS (
        SELECT *
        FROM TD_XGBoostPredict(
            ON df_test AS inputtable PARTITION BY ANY
            ON xgb_model AS modeltable DIMENSION ORDER BY task_index, tree_num, iter, class_num, tree_order
            USING
                IdColumn('uid')
                ModelType('classification')
                Accumulate('Failure_Type')
        ) AS dt
    ) WITH DATA;
'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE xgb_predict_out;')
    eng.execute(query)

<p style = 'font-size:16px;font-family:Arial'>Next, we'll use the TD_ClassificationEvaluator function to evaluate the trained xgboost model on test data. This will let us know how well our model has performed on unseen data.</p>

In [ ]:
query = '''
    SELECT *
    FROM TD_CLASSIFICATIONEVALUATOR(
        ON (
        SELECT CAST("Failure_Type" AS INTEGER) AS "Failure_Type", prediction FROM xgb_predict_out) AS InputTable
        OUT TABLE OutputTable(additional_metrics)
        USING
        Labels(0, 1, 2, 3, 4)
        ObservationColumn('Failure_Type')
        PredictionColumn('Prediction')
    ) AS dt1
    ORDER BY 1, 2, 3;
'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE additional_metrics;')
    eng.execute(query)

In [ ]:
pd.read_sql('select * from additional_metrics', eng)

In [ ]:
# Load prediction results into a pandas dataframe
xgb_result = DataFrame(in_schema('demo_user', 'xgb_predict_out')).to_pandas().reset_index()

# Calculate confusion matrix
cm = confusion_matrix(xgb_result['Prediction'], xgb_result['Failure_Type'])

# Plot confusion matrix using ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

# Create figure and axes objects
fig, ax = plt.subplots(figsize=(8, 8))

# Plot confusion matrix
disp.plot(ax=ax, cmap='Blues', colorbar=False)

# Set title and axis labels
plt.title('XGBoost Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Set x and y ticks with labels and rotation
plt.xticks(ticks=[0, 1, 2, 3, 4], labels=['No Failure', 'Heat Dissipation Failure', 'Power Failure', 'Overstrain Failure', 'Tool Wear Failure'], rotation=45)
plt.yticks(ticks=[0, 1, 2, 3, 4], labels=['No Failure', 'Heat Dissipation Failure', 'Power Failure', 'Overstrain Failure', 'Tool Wear Failure'])

# Add text to the plot to show the actual values of the confusion matrix
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, f'{cm[i, j]}', ha='center', va='center', color='white' if cm[i, j] > cm.max()/2 else 'black')

# Show the plot
plt.show()


<p style = 'font-size:16px;font-family:Arial'>The above metrics show that our model performs well on the multi-class classification test dataset.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>By implementing a predictive maintenance solution, <b>Y-Machine</b> can significantly reduce their machine downtime and maintenance costs while improving their overall production efficiency. The solution will allow them to proactively schedule maintenance and avoid costly breakdowns, reducing the need for emergency repairs and improving the reliability of their machines. With real-time reporting and analytics, <b>Y-Machine</b> can make data-driven decisions about their maintenance strategy, improving the performance of their machines and increasing their overall productivity.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>9. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
eng.execute('DROP TABLE df_train;')

In [ ]:
eng.execute('DROP TABLE df_test;')

In [ ]:
eng.execute('DROP TABLE xgb_model;')

In [ ]:
eng.execute('DROP TABLE xgb_out;')

In [ ]:
eng.execute('DROP TABLE xgb_predict_out;')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_PredictiveMaintenance');"        # Takes 5 seconds

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Dataset:</b>

- `UID`: Unique identifier ranging from 1 to 10000
- `Product_ID`: Unique Product ID consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number
- `Type`: Consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number
- `Air_temperature`: Generated using a random walk process later normalized to a standard deviation of 2 K around 300 K
- `Process_temperature`: Generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K
- `Rotational_speed`: Calculated from a power of 2860 W, overlaid with a normally distributed noise
- `Torque`: Torque values are normally distributed around 40 Nm with a Ïƒ = 10 Nm and no negative values
- `Tool_wear`: The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process
- `Target`: If the machine failed or not (boolean)
- `Failure_Type`: Type of failure -
                            No Failure,
                            Heat Dissipation Failure,
                            Power Failure,
                            Overstrain Failure,
                            Tool Wear Failure

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>